<a href="https://colab.research.google.com/github/sseongeun/CiWinterProject/blob/main/my_resnet_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

from keras.utils.np_utils import to_categorical
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.datasets import cifar10



import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler


In [ ]:
#plt.figure(figsize=(1,1))  # 너비와 높이를 모두 3인치로 설정
#plt.imshow(x_train[0],cmap='binary')
#plt.show


In [ ]:
#resnet 모델 구현 코드
def ResnetBlock(num_filters, input_layer, strides=(1, 1)):
    shortcut = input_layer
    if strides != (1, 1) or input_layer.shape[-1] != num_filters:
        shortcut = layers.Conv2D(num_filters, (1, 1), strides=strides, padding='valid', kernel_initializer='he_normal')(input_layer)

    x = layers.Conv2D(num_filters, (3, 3), padding='same', strides=strides, kernel_initializer='he_normal')(input_layer)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    x = layers.Conv2D(num_filters, (3, 3), padding='same', kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Add()([shortcut, x])
    x = layers.Activation('relu')(x)
    
    return x

def ResNet(input_shape, num_classes=10):
    inputs = layers.Input(shape=input_shape)
    
    x = layers.Conv2D(16, (3, 3), padding='same', kernel_initializer='he_normal')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    num_filters = 16 
    for stack in range(3):
        for block in range(18):  # 6n layers, n=18 for ResNet110
            strides = (1, 1)
            if stack > 0 and block == 0:  # first layer but not first stack
                strides = (2, 2)  # downsample
            x = ResnetBlock(num_filters, x, strides)
        num_filters *= 2
    
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    return model


# Prepare CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Normalize pixel values to [0, 1] and subtract per-pixel mean
x_train = x_train.astype('float32')# / 255
x_test = x_test.astype('float32')# / 255
x_train -= x_train.mean(axis=0)
x_test -= x_test.mean(axis=0)

# Convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# # Add padding of 4 pixels on each side
# x_train_padded = np.pad(x_train, ((0, 0), (4, 4), (4, 4), (0, 0)), mode='reflect')

# Data augmentation
datagen = ImageDataGenerator(
    horizontal_flip=True,
    width_shift_range=8,
    height_shift_range=8
)

# Instantiate and compile the model
model = ResNet((32, 32, 3), 10)



170498071/170498071 [==============================] - 2s 0us/step


In [ ]:
# Learning rate schedule
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=x_train.shape[0]//256 * 20, # 200 epochs
    decay_rate=0.1, # decay to 0.1 of initial learning rate
    staircase=True)

sgd = tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


In [ ]:
# Fit the model with batch size 256
model.fit(datagen.flow(x_train, y_train, batch_size=256), 
          steps_per_epoch=len(x_train) // 256, 
          epochs=30, 
          validation_data=(x_test, y_test))

Epoch 1/20
195/195 [==============================] - 119s 337ms/step - loss: 2.2077 - accuracy: 0.2547 - val_loss: 2.0998 - val_accuracy: 0.2590
Epoch 2/20
195/195 [==============================] - 64s 329ms/step - loss: 1.6107 - accuracy: 0.4024 - val_loss: 1.4864 - val_accuracy: 0.4638
Epoch 3/20
195/195 [==============================] - 64s 325ms/step - loss: 1.4475 - accuracy: 0.4697 - val_loss: 1.4843 - val_accuracy: 0.4563
Epoch 4/20
195/195 [==============================] - 63s 324ms/step - loss: 1.3290 - accuracy: 0.5152 - val_loss: 1.5013 - val_accuracy: 0.4974
Epoch 5/20
195/195 [==============================] - 64s 330ms/step - loss: 1.2352 - accuracy: 0.5517 - val_loss: 1.3970 - val_accuracy: 0.5190
Epoch 6/20
195/195 [==============================] - 64s 329ms/step - loss: 1.1480 - accuracy: 0.5833 - val_loss: 1.1845 - val_accuracy: 0.5928
Epoch 7/20
195/195 [==============================] - 63s 323ms/step - loss: 1.0816 - accuracy: 0.6127 - val_loss: 1.0666 - val_a

In [ ]:
model.save('resnet_model.h5')

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 16)   448         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 16)  64          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 32, 32, 16)   0           ['batch_normalization[0][0]']

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Grad-CAM을 위한 함수 정의
def grad_cam(model, x, layer_name):
    # 모델의 특정 레이어의 출력과 gradient를 얻기 위한 함수 정의
    grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(layer_name).output, model.output])

    # 입력 이미지에 대한 예측값과 gradient 계산
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(x)
        loss = predictions[:, tf.argmax(predictions[0])]

    # gradient 계산
    gradients = tape.gradient(loss, conv_outputs)[0]

    # 마지막 레이어의 필터별 gradient의 평균을 계산하여 중요도로 사용
    weights = tf.reduce_mean(gradients, axis=(0, 1))

    # 각 채널의 활성화 맵에 대한 가중합
    cam = tf.reduce_sum(tf.multiply(weights, conv_outputs), axis=-1)

    # 활성화 맵을 0~1로 정규화
    cam = np.maximum(cam, 0) / np.max(cam)

    return cam


# 이미지 전처리 함수
def preprocess_input(x):
    x = x.astype('float32')
    x /= 255.0
    x -= 0.5
    x *= 2.0
    return x


# CIFAR-10 데이터셋 로드
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# 이미지 전처리 및 데이터셋 준비
x_test = preprocess_input(x_test)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

# ResNet 모델 로드
model = tf.keras.models.load_model('resnet_model.h5')  # ResNet 모델 파일 경로로 수정

# Grad-CAM을 시각화할 이미지 선택
image_index = 0  # 예시로 첫 번째 이미지 사용
x = np.expand_dims(x_test[image_index], axis=0)
y_true = np.argmax(y_test[image_index])

# Grad-CAM 계산
cam = grad_cam(model, x, 'conv2d_110')


In [ ]:

# 원본 이미지 가져오기
image = x_test[image_index]
image = (image + 1) / 2.0  # 이미지 후처리 역과정



In [ ]:
# Grad-CAM을 원본 이미지에 적용하여 시각화
cam = cv2.resize(cam, (image.shape[1], image.shape[0]))
heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)

error: ignored

In [ ]:


# Grad-CAM을 원본 이미지에 적용하여 시각화
cam = cv2.resize(cam, (image.shape[1], image.shape[0]))
cam = cam.astype(np.uint8)  # 데이터 타입 변환
cam = cv2.cvtColor(cam, cv2.COLOR_GRAY2BGR)  # 단일 채널 이미지를 3채널 이미지로 변환

heatmap = cv2.applyColorMap(cam, cv2.COLORMAP_JET)
heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)

# 원본 이미지와 Grad-CAM 시각화
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.imshow(image)
plt.axis('off')
plt.title('Original Image')

plt.subplot(1, 2, 2)
plt.imshow(image)

error: ignored